In [1]:
import os, time, torch

In [2]:
%run ../utilities/curl_utilities.ipynb
%run ../utilities/postgres_utilities.ipynb

In [3]:
model_name = "GritLM/GritLM-7B"
number_data_rows = 100

In [5]:
batch_size = 100

In [6]:
# Use a pipeline as a high-level helper
from transformers import pipeline, AutoTokenizer

# Load the tokenizer for your model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set the padding_side parameter (can be 'left' or 'right')
tokenizer.padding_side = 'left'  # This sets padding to the left side

#Note that model will not load without reducing precision to float16
pipe = pipeline("text-generation", model=model_name, tokenizer=tokenizer, trust_remote_code=False, torch_dtype=torch.float16,
               batch_size=batch_size)

/opt/app-root/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [01:06<00:00, 22.25s/it]
Device set to use cuda:0


In [7]:
sql = f"select description from film limit {number_data_rows}"
film_descriptions = execute_sql_results(sql)

In [8]:
test_description = "Starting batch generative test using local model"
print(test_description)
print(len(test_description) * "-") 
tic = time.perf_counter()
print("Texts generated ...", end=" ")
messages = list()
for index, film in enumerate(film_descriptions):
    description = film[0]
    prompt = f"Expand this movie description: {description}"
    messages.append([{"role": "user", "content": prompt}])

start_offset = 0
end_offset = batch_size
question_response_pairs = list()
while start_offset < number_data_rows:
    print(start_offset, end_offset, end=" ")
    responses = pipe(messages[start_offset:end_offset], max_new_tokens=50)

    for r in responses:
        original_prompt = r[0]['generated_text'][0]['content']
        generated_text = r[0]['generated_text'][1]['content']
        assert len(generated_text) > 0
        question_response_pairs.append(f"{original_prompt}:{generated_text}")

    start_offset = end_offset
    end_offset = end_offset + batch_size
    if end_offset > number_data_rows:
        end_offset = number_data_rows

assert len(question_response_pairs) == number_data_rows

toc = time.perf_counter()
print("")
print(f"Time to generate text for {number_data_rows} rows: {toc - tic:0.4f} seconds")
print(len(test_description) * "-") 

Starting batch generative test using local model
------------------------------------------------
Texts generated ... 0 100 
Time to generate text for 100 rows: 6.4079 seconds
------------------------------------------------


In [10]:
close_db()